In [1]:
#Para evitar la descarga local del archivo, conecto la API de Kaggle, para luego descargar el dataset y guardarlo en el drive.
! pip install -q kaggle

In [4]:
#Creacion del directorio
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/


In [6]:
#Cambio los permisos para que me permita leer las credenciales
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
#Pruebo que funcione, listando los datasets
! kaggle datasets list


ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                                 Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          30818        844  1.0              
tawfikelmetwally/automobile-dataset                                   Car information dataset                               6KB  2023-05-28 18:26:48           3232         88  0.9411765        
fatihb/coffee-quality-data-cqi                                        Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           5559        114  1.0              
mohithsairamreddy/salary-data 

In [2]:
#Descargo uno de los datasets
! kaggle datasets download -d utkarshx27/starbucks-nutrition

  0%|                                               | 0.00/1.81k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 1.81k/1.81k [00:00<00:00, 629kB/s]


In [32]:
!pip install psycopg2-binary

In [33]:
import os
import psycopg2


from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col

In [34]:
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.5.2.jar"


In [35]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

In [36]:
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [37]:
env = os.environ

In [38]:
#Dataframe
df = spark.read.csv("/home/coder/working_dir/starbucks.csv", sep=",", header=True)


In [39]:
df.show()

+---+--------------------+--------+---+----+-----+-------+------+
|_c0|                item|calories|fat|carb|fiber|protein|  type|
+---+--------------------+--------+---+----+-----+-------+------+
|  1|        8-Grain Roll|     350|  8|  67|    5|     10|bakery|
|  2|   Apple Bran Muffin|     350|  9|  64|    7|      6|bakery|
|  3|       Apple Fritter|     420| 20|  59|    0|      5|bakery|
|  4|     Banana Nut Loaf|     490| 19|  75|    4|      7|bakery|
|  5|Birthday Cake Min...|     130|  6|  17|    0|      0|bakery|
|  6|   Blueberry Oat Bar|     370| 14|  47|    5|      6|bakery|
|  7|     Blueberry Scone|     460| 22|  61|    2|      7|bakery|
|  8|Bountiful Blueber...|     370| 14|  55|    0|      6|bakery|
|  9|   Butter Croissant |     310| 18|  32|    0|      5|bakery|
| 10|       Cheese Danish|     420| 25|  39|    0|      7|bakery|
| 11|Chocolate Chunk C...|     380| 17|  51|    2|      4|bakery|
| 12|Chocolate Cinnamo...|     320| 12|  53|    3|      6|bakery|
| 13| Choc

In [41]:
#Le renombro una columna ID a la tabla
df_to_write = df.withColumnRenamed("_c0","id")
df_to_write.printSchema()
df_to_write.show()

root
 |-- id: string (nullable = true)
 |-- item: string (nullable = true)
 |-- calories: string (nullable = true)
 |-- fat: string (nullable = true)
 |-- carb: string (nullable = true)
 |-- fiber: string (nullable = true)
 |-- protein: string (nullable = true)
 |-- type: string (nullable = true)

+---+--------------------+--------+---+----+-----+-------+------+
| id|                item|calories|fat|carb|fiber|protein|  type|
+---+--------------------+--------+---+----+-----+-------+------+
|  1|        8-Grain Roll|     350|  8|  67|    5|     10|bakery|
|  2|   Apple Bran Muffin|     350|  9|  64|    7|      6|bakery|
|  3|       Apple Fritter|     420| 20|  59|    0|      5|bakery|
|  4|     Banana Nut Loaf|     490| 19|  75|    4|      7|bakery|
|  5|Birthday Cake Min...|     130|  6|  17|    0|      0|bakery|
|  6|   Blueberry Oat Bar|     370| 14|  47|    5|      6|bakery|
|  7|     Blueberry Scone|     460| 22|  61|    2|      7|bakery|
|  8|Bountiful Blueber...|     370| 14|  

In [42]:
os.environ['AWS_REDSHIFT_USER']='danisa_altamirano9_coderhouse'
os.environ['AWS_REDSHIFT_PASSWORD']='JNl7RO87TW'
os.environ['AWS_REDSHIFT_HOST']='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
os.environ['AWS_REDSHIFT_PORT']='5439'
os.environ['AWS_REDSHIFT_DBNAME']='data-engineer-database'
os.environ['AWS_REDSHIFT_SCHEMA']='danisa_altamirano9_coderhouse'

In [43]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD']
)

In [44]:
cursor = conn.cursor()

cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.entregable1 (
    id int,
    item varchar, 
    calories int, 
    fiber int, 
    protein int, 
    type varchar);
""")
conn.commit()
cursor.close()
print("Table created!")

Table created!


In [233]:
cursor = conn.cursor()
cursor.execute(f"""
SELECT
  distinct tablename
FROM
  PG_TABLE_DEF
WHERE
  schemaname = '{env['AWS_REDSHIFT_SCHEMA']}';
""")
# resultado = cursor.fetchall()
print(", ".join(map(lambda x: x[0], cursor.fetchall())))
cursor.close()

In [ ]:
df_to_write.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.entregable1") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

<b><h1>ENTREGABLE NRO 2<h1><b>

In [253]:
from sqlalchemy import create_engine
import pandas as pd



In [261]:
#Renombre el nombre de la columna type
df2.rename(columns = {'type':'category'}, inplace = True)

In [240]:
cursor = conn.cursor()

cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.entregable1_v2 (
    id int,
    item varchar, 
    calories int, 
    fiber int, 
    protein int, 
    type varchar);
""")
conn.commit()
cursor.close()
print("Table created!")

Table created!


In [234]:
df2

,item,calories,fat,carb,fiber,protein,category
1,8-Grain Roll,350,8.0,67,5,10,bakery
2,Apple Bran Muffin,350,9.0,64,7,6,bakery
3,Apple Fritter,420,20.0,59,0,5,bakery
4,Banana Nut Loaf,490,19.0,75,4,7,bakery
5,Birthday Cake Mini Doughnut,130,6.0,17,0,0,bakery
...,...,...,...,...,...,...,...
73,Tarragon Chicken Salad Sandwich,420,13.0,46,6,32,sandwich
74,Turkey & Swiss Sandwich,390,13.0,36,2,34,sandwich
75,Greek Yogurt Honey Parfait,300,12.0,44,0,8,parfait
76,Peach Raspberry Yogurt Parfait,300,4.0,57,3,10,parfait


In [243]:
df = df2

In [249]:
cursor = conn.cursor()
df2.

In [246]:
df_to_write.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.entregable1_v2") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [227]:
#Levanto el DF con la libreria descargada de PANDAS
df2=pd.read_csv("/home/coder/working_dir/starbucks.csv", index_col=[0])


In [203]:
#Controlar que no haya duplicados
df2.duplicated()


1     False
2     False
3     False
4     False
5     False
      ...  
73    False
74    False
75    False
76    False
77    False
Length: 77, dtype: bool

In [144]:
#Ordeno el dataframe de acuerdo a los valores de la columna TYPE
print(df2.sort_values('type'))

                                  item  calories   fat  carb  fiber  protein   
1                         8-Grain Roll       350   8.0    67      5       10  \
24              Maple Oat Pecan Scone        440  18.0    59      3        8   
25                   Marble Pound Cake       350  13.0    54      0        6   
26               Marshmallow Dream Bar       210   4.0    43      0        0   
27                         Morning Bun       350  16.0    45      2        6   
..                                 ...       ...   ...   ...    ...      ...   
70                  Ham & Swiss Panini       360   9.0    43      2       28   
71  Roasted Tomato & Mozzarella Panini       390  18.0    44      3       15   
72            Roasted Vegetable Panini       350  12.0    48      4       13   
73     Tarragon Chicken Salad Sandwich       420  13.0    46      6       32   
74             Turkey & Swiss Sandwich       390  13.0    36      2       34   

        type  
1     bakery  
24    bak

In [145]:
print(df2.groupby('type').get_group('bakery'))

                                             item  calories   fat  carb   
1                                    8-Grain Roll       350   8.0    67  \
2                               Apple Bran Muffin       350   9.0    64   
3                                   Apple Fritter       420  20.0    59   
4                                 Banana Nut Loaf       490  19.0    75   
5                     Birthday Cake Mini Doughnut       130   6.0    17   
6                               Blueberry Oat Bar       370  14.0    47   
7                                 Blueberry Scone       460  22.0    61   
8                      Bountiful Blueberry Muffin       370  14.0    55   
9                               Butter Croissant        310  18.0    32   
10                                  Cheese Danish       420  25.0    39   
11                         Chocolate Chunk Cookie       380  17.0    51   
12                       Chocolate Cinnamon Bread       320  12.0    53   
13                       

In [146]:
#Agrupo los datos por los distintos tipos de productos
df_by_type = df2.groupby(by = "type")

In [147]:
df_by_type.groups

{'bakery': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'bistro box': [42, 43, 44, 45, 46, 47, 48, 49], 'hot breakfast': [59, 60, 61, 62, 63, 64, 65, 66], 'parfait': [75, 76, 77], 'petite': [50, 51, 52, 53, 54, 55, 56, 57, 58], 'salad': [67], 'sandwich': [68, 69, 70, 71, 72, 73, 74]}

In [148]:
#Cuantos productos hay por categoria
df_by_type.size()

type
bakery           41
bistro box        8
hot breakfast     8
parfait           3
petite            9
salad             1
sandwich          7
dtype: int64

In [158]:
#Producto con valores generales mas altos
df2.max()

item        Zucchini Walnut Muffin 
calories                        500
fat                            28.0
carb                             80
fiber                             7
protein                          34
type                       sandwich
dtype: object

In [188]:
#Seleccionar las columnas que necesito
df3 = df2.columns.isin(['calories','item','type'])

In [189]:
select = df2.columns[df3]

In [200]:
#producto con mayor calorias
df2[select].max()

item        Zucchini Walnut Muffin 
calories                        500
type                       sandwich
dtype: object

In [201]:
#producto con menor calorias
df2[select].min()

item        8-Grain Roll
calories              80
type              bakery
dtype: object

In [196]:
df4 = df2.columns.isin(['fiber','item','type'])

In [197]:
select1 = df2.columns[df4]

In [199]:
#producto con mayor fibras
df2[select1].max()

item     Zucchini Walnut Muffin 
fiber                          7
type                    sandwich
dtype: object